In [1]:
import pandas as pd
import urllib.request
import json
import time

# Define the API Key and Base URLs
API_KEY = 'a231ab0915a3d2f61aa1880fb95dec1e'
COUNTY_BASE_URL = 'https://api.stlouisfed.org/fred/release/tables'
OBSERVATIONS_URL = 'https://api.stlouisfed.org/fred/series/observations'

# Function to fetch population data for a given series_id and year
def fetch_population(series_id, year):
    query_url = (
        f"{OBSERVATIONS_URL}?"
        f"series_id={series_id}&"
        f"api_key={API_KEY}&"
        f"file_type=json"
    )
    try:
        response = urllib.request.urlopen(query_url)
        response_data = json.loads(response.read())
        
        # Extract the value for the specified year
        if 'observations' in response_data:
            for observation in response_data['observations']:
                if observation['date'].startswith(year):
                    return float(observation['value'])
    except urllib.error.HTTPError as e:
        if e.code == 429:  # Handle rate limit
            print(f"Rate limit hit. Retrying after delay...")
            time.sleep(5)  # Delay for 5 seconds
            return fetch_population(series_id, year)  # Retry
        else:
            print(f"HTTP Error for {series_id}, year {year}: {e}")
    except Exception as e:
        print(f"Error fetching data for {series_id}, year {year}: {e}")
    return None

# Parameters for the request
params = {
    'release_id': 119,
    'element_id': 163194,  # Ensure this is correct for California
    'api_key': API_KEY,
    'file_type': 'json',
    'od': '2022-01-01'  # Observation date for 2022
}

query_url = (
    f"{COUNTY_BASE_URL}?"
    f"release_id={params['release_id']}&"
    f"element_id={params['element_id']}&"
    f"api_key={params['api_key']}&"
    f"file_type={params['file_type']}&"
    f"od={params['od']}"
)

# Main script to fetch data
try:
    response = urllib.request.urlopen(query_url)
    response_data = json.loads(response.read())
    
    # Extract the 'elements' key from the response
    if 'elements' in response_data:
        elements = response_data['elements']
        
        # Prepare data
        data = []
        years = [str(year) for year in range(2011, 2023)]  # Years 2011–2022
        for key, value in elements.items():
            county_name = value['name']
            series_id = value['series_id']
            
            # Fetch population for each year
            for year in years:
                population = fetch_population(series_id, year)
                if population is not None:
                    data.append({'County Name': county_name, 'Year': year, 'Population': population})
                time.sleep(0.5)  # Delay of 0.5 seconds between requests
        
        # Create a DataFrame
        df = pd.DataFrame(data)
        print("DataFrame:")
        print(df.head())  # Display the resulting DataFrame
        
    else:
        print("No 'elements' key found in the response. Check the response structure.")
except Exception as e:
    print(f"An error occurred: {e}")


DataFrame:
      County Name  Year  Population
0  Alameda County  2011    1530.893
1  Alameda County  2012    1553.729
2  Alameda County  2013    1579.508
3  Alameda County  2014    1607.638
4  Alameda County  2015    1634.326


In [2]:
#Multiply population column by 1000 to yield the true population of each county
df['Population'] = (df['Population'] * 1000).astype(int)
df.head()

,County Name,Year,Population
0,Alameda County,2011,1530893
1,Alameda County,2012,1553729
2,Alameda County,2013,1579508
3,Alameda County,2014,1607638
4,Alameda County,2015,1634326


In [3]:
df.to_csv('Cal_County_Pop_2011-2022.csv',index=False)